In [1]:
# Básicos:
import pandas as pd
import numpy as np
import json
import re
from collections import Counter
from unidecode import unidecode
from dotenv import load_dotenv
import os

# Tiempo
import datetime
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

# Texto
import nltk
from langdetect import detect

# API
import googlemaps

# Extra
import ipywidgets as widgets
from IPython.display import display
import tqdm
import pickle

# 1. Limpieza

### 1.1. Añadimos columnas

In [2]:
def add_columns (df, columns):
    for col in columns:
        if col not in df.columns:
            df[col] = np.nan
    return df

In [3]:
def convert_to_lowercase(valor):
    if isinstance(valor, str):
        return valor.lower()
    return valor

### 1.2. Quitamos todos los caracteres especiales

In [4]:
def remove_special_characters(df, exclude_columns=['herramientas', 'url']):
    for column in df.columns:
        if column not in exclude_columns:
            df[column] = df[column].replace(r'[^\w\s/ñÑ-]', '', regex=True)
            df[column] = df[column].map(lambda x: re.sub(r'[áéíóúüàèìòù]', lambda m: unidecode(m.group()), str(x)))
    return df

### 1.3. Convertimos las columnas de fechas a DateTime

In [5]:
def date_matching(date):
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})|(\d+)\s*(minutos|hora|dia|semana|mes)', date)
    
    if match:
        if match.group(1):
            day = int(match.group(1))
            month = int(match.group(2))
            year = int(match.group(3))
            return relativedelta(day=day, month=month, year=year)
        else:
            cantidad = int(match.group(4))
            unidad = match.group(5)
        
            if unidad == 'minutos':
                return relativedelta(minutes=cantidad)
            elif unidad == 'hora':
                return relativedelta(hours=cantidad)
            elif unidad == 'dia':
                return relativedelta(days=cantidad)
            elif unidad == 'semana':
                return relativedelta(weeks=cantidad)
            elif unidad == 'mes':
                return relativedelta(months=cantidad)
    
    return relativedelta()

### 1.4. Extraemos los datos numericos, salarios, beneficios y las herramientas

- ### Datos numéricos:

In [6]:
def extract_numbers(text):
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    else:
        return None

- ### Salarios:

In [7]:
def encontrar_salarios(texto):
    patron = re.compile(r'\b\d{2,3}[kK]\s*-\s*\d{2,3}[kK]\b|\b\d{5,6}\s*-\s*\d{5,6}\b')
    matches = patron.findall(texto)
    return matches

In [8]:
def rellenar_salarios(df):
    for idx, row in df.iterrows():
        descripcion = row["descripcion"]
        salario = row["salario"]
        if pd.isna(salario):
            if re.search(r'\b\d{2,3}[kK]\s*-\s*\d{2,3}[kK]\b|\b\d{5,6}\s*-\s*\d{5,6}\b', descripcion):
                df.loc[idx, "salario"] = encontrar_salarios(descripcion)[0]

- ### Herramientas:

In [9]:
def tool_cleaner(df, numero= 1000):

    herramientas_totales = ""

    for herramientas in df["herramientas"]:
        herramientas = str(herramientas)
        herramientas_limpias = (re.sub(r'[^\w\s+]', '', herramientas) + " ")
        herramientas_totales += re.sub(r'[áéíóúüàèìòù]', lambda m: unidecode(m.group()), herramientas_limpias)
    
    # Tokenizamos las herramientas totales:
    tokens = nltk.word_tokenize(text = herramientas_totales, language = "spanish")
    stopwords = nltk.corpus.stopwords.words("spanish")

    # Quitamos stopwords:
    clean_tokens = []
    for token in tokens:
        if token not in stopwords:
            clean_tokens.append(token)

    herramientas_sin_stopwords = " ".join(clean_tokens)
      
    tool_count = Counter()
    for herramienta in herramientas_sin_stopwords.split():    
        tool_count[herramienta] += 1

    common_tools = tool_count.most_common(numero)
    
    return common_tools

In [10]:
def buscar_herramienta(texto, herramienta):
    if re.search(r'\b' + herramienta + r'\b', texto):
        return True
    return False

- ### Beneficios:

In [11]:
def buscar_beneficios(texto, lista_beneficios):
    
    beneficios = 0
    for beneficio in lista_beneficios:
        if re.search(r'\b' + beneficio + r'\b', texto):
            beneficios += 1
            if beneficios > 1:
                return True
            
    return False

### 1.5. Asignamos ubicaciones

In [12]:
def asignar_ubicaciones(merged_df, lista_ubicaciones):
    for index, row in merged_df.iterrows():
        descripcion = row['descripcion']
        ubicacion = row['ubicacion']

        if pd.isna(ubicacion):
            found_match = False
            for sublista in lista_ubicaciones:
                for label in sublista:
                    label_cleaned = re.sub(r'[^\w\s]', '', str(label).lower())
                    if label_cleaned in descripcion.lower():
                        merged_df.at[index, 'ubicacion'] = label
                        found_match = True
                        break
                if found_match:
                    break

            if not found_match:
                merged_df.at[index, 'ubicacion'] = 'españa'

### 1.6. Procesamos todos los Df

In [13]:
def preprocess_dataframes(df_dic, columns, columnas_numericas):
    for key, df in df_dic.items():
        df = add_columns(df, columns)
        df = df.dropna(subset=['titulo', 'empresa', 'descripcion'], thresh=2)
        df = df.map(convert_to_lowercase)
        df = remove_special_characters(df)
        df[columnas_numericas] = df[columnas_numericas].map(extract_numbers)
        df[columnas_numericas] = df[columnas_numericas].fillna(0).astype(int)
        df = df.sort_values(by='fecha_scrapeo')
        df = df.drop_duplicates(subset=df.columns.difference(['fecha', 'solicitudes', 'fecha_scrapeo', 'url', 'portal']), keep='first')
        df_dic[key] = df

# 2. Mapping

### 2.1. Funciones del mapeo

In [14]:
def map_titulo(text, mapping_titulo):
    for pattern, label in mapping_titulo.items():
        if re.match(pattern, text, re.IGNORECASE):
            return label
    return text

In [15]:
def map_presencialidad(text, mapping_presencialidad):
    for pattern, label in mapping_presencialidad.items():
        if re.match(pattern, text, re.IGNORECASE):
            return label
    return 'no especificado'

In [16]:
def map_jornada(text, mapping_jornada):
    for pattern, label in mapping_jornada.items():
        if re.match(pattern, text, re.IGNORECASE):
            return label
    return 'no especificado'

In [17]:
def map_contrato(text, mapping_contrato):
    for pattern, label in mapping_contrato.items():
        if re.match(pattern, text, re.IGNORECASE):
            return label
    return 'no especificado'

# 3. Apis

## 3.1. GoogleMaps

- ### Ajuste para la api

In [18]:
def obtener_dato_ubicacion(palabra_localizador, geocode_result):
    try:
        dato = [elemento['long_name'] for elemento in geocode_result if palabra_localizador in elemento['types']][0]
        return dato
    except:
        return np.nan

In [19]:
def agregar_galicia(row):
    if 'galicia' in str(row['ubicacion']):
        return 'Galicia', 'España'
        
    elif pd.isnull(row['comunidad']):
        provincias_galicia = ['A Coruña', 'La Coruña', 'Pontevedra', 'Lugo', 'Ourense']
        if row['provincia'] in provincias_galicia:
            return 'Galicia', 'España'
            
    return row['comunidad'], row['pais']

In [ ]:
def obtener_dato_ubicacion(tipo_dato, address_components):
    for component in address_components:
        if tipo_dato in component['types']:
            return component['long_name']
    return np.nan

In [ ]:
def obtener_ubicaciones(df, gmaps, MAPS_API_KEY):
    localidades = []
    provincias = []
    comunidades = []
    paises = []
    latitudes = []
    longitudes = []
    ubicaciones_fallidas = []

    for idx, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        ubicacion = row['ubicacion']
            
        geocode_result = gmaps.geocode(ubicacion, language='es')

        if geocode_result:
            localidad = obtener_dato_ubicacion('locality', geocode_result[0]['address_components'])
            provincia = obtener_dato_ubicacion('administrative_area_level_2', geocode_result[0]['address_components'])
            comunidad = obtener_dato_ubicacion('administrative_area_level_1', geocode_result[0]['address_components'])
            pais = obtener_dato_ubicacion('country', geocode_result[0]['address_components'])
            latitud = geocode_result[0]['geometry']['location']['lat']
            longitud = geocode_result[0]['geometry']['location']['lng']
            print(f'Localidad: {localidad}, Provincia: {provincia}, Comunidad: {comunidad}, Pais: {pais} Latitud: {latitud}, Longitud: {longitud}')
        else:
            localidad, provincia, comunidad, pais, latitud, longitud = (np.nan,) * 6
            ubicaciones_fallidas.append(ubicacion)
            print(f'No se encontraron resultados en: {ubicacion}')

        localidades.append(localidad)
        provincias.append(provincia)
        comunidades.append(comunidad)
        paises.append(pais)
        latitudes.append(latitud)
        longitudes.append(longitud)

    df['localidad'] = localidades
    df['provincia'] = provincias
    df['comunidad'] = comunidades
    df['pais'] = paises
    df['latitud'] = latitudes
    df['longitud'] = longitudes

    return df, ubicaciones_fallidas